# Robustness Noise 
In order to understand the results better from the testing so far, in particular the plots, we want to analyse how noise influences our computations.
This serves the overall goal of improving our method with noisy data.
Following questions arise:

How can we bound the finite difference method?

How stable is the SVD to noise?

We only consider noise which is normally distributed? Does this influence our result?

What is the relation between the highest and the lowest singular value? Do they grow/shrink in a similar way when confronted with noise?

## SVD Robustness Noise
There is a theorem and a corollary:

Theorem 2.18 (Mirsky). If $A,E \in K^{IxI}$ are two arbitrary matrices, then $\sqrt{\sum_{k=1}^{r}|\sigma_k(A+E) - \sigma_k(A)|} \leq \lVert E \rVert_F$

Corollary:
If $A,E \in K^{IxI}$ are two arbitrary matrices, then $\forall k \ |\sigma_k(A+E) - \sigma_k(A)| \leq \lVert E \rVert$

1. What is the spectral and Frobenius norm of random noise matrices?

In [165]:
import numpy as np
from numpy.linalg import matrix_rank, svd
from test_data import experiment_data,add_noise
import pysindy as ps

In [166]:
rows,cols = 3,3 #let rows be more than columns
min_value,max_value = -10,10
matrix = np.random.uniform(min_value, max_value, size=(rows, cols))
# Set col 2 equal to col 1
alpha = np.random.rand()
matrix[:, 1] = alpha*matrix[:,2]
print(matrix)

[[ 5.51247314 -7.97141108 -8.65231561]
 [-1.95473907  6.40081464  6.94756146]
 [ 4.72324461  6.75677805  7.33393067]]


In [167]:
print(matrix_rank(matrix),svd(matrix, compute_uv=False))
print(np.linalg.norm(matrix))
print(np.linalg.norm(matrix,ord=2))

2 [1.82167427e+01 7.19397514e+00 1.02472235e-15]
19.585785446432766
18.21674266443416


Add noise

In [168]:
#np.random.seed(12)
target_noise=1e-3
var = target_noise * np.sqrt(np.mean(np.square(matrix)))
noise = np.random.normal(0, var, size=matrix.shape)
matrix_noise = matrix + noise
print(f"Added Gaussian noise with variance {var}")

Added Gaussian noise with variance 0.006528595148810923


In [169]:
print("Noise Matrix")
#print(noise)
print(f"Matrix rank: {matrix_rank(noise)}, SVD: {svd(noise, compute_uv=False)}")
print(f"Frobenius norm:\t {np.linalg.norm(noise)}")
print(f"Spectral norm:\t {np.linalg.norm(noise,ord=2)}")

Noise Matrix
Matrix rank: 3, SVD: [0.01939426 0.00693597 0.00419482]
Frobenius norm:	 0.02102002855355578
Spectral norm:	 0.019394261301622036


Print SVD and matrix rank with additional noise

In [171]:
noise_levels = [0]+[10**(-10+i) for i in range(0,10)]
print("Noise level \t \t matrix rank \t svd \t \t \t \t \t \t \t Frobenius \t \t \t Spectral")
for target_noise in noise_levels:
    var = target_noise * np.sqrt(np.mean(np.square(matrix)))
    noise = np.random.normal(0, var, size=matrix.shape)
    matrix_noise = matrix + noise
    print(f" {target_noise}      \t \t {matrix_rank(matrix_noise)} \t \t {svd(matrix_noise, compute_uv=False)}     \t {np.linalg.norm(noise)}      \t {np.linalg.norm(noise,ord=2)}")

Noise level 	 	 matrix rank 	 svd 	 	 	 	 	 	 	 Frobenius 	 	 	 Spectral
 0      	 	 2 	 	 [1.82167427e+01 7.19397514e+00 1.02472235e-15]     	 0.0      	 0.0
 1e-10      	 	 3 	 	 [1.82167427e+01 7.19397513e+00 4.03147296e-10]     	 1.7488295677824028e-09      	 1.5414851495545185e-09
 1e-09      	 	 3 	 	 [1.82167427e+01 7.19397513e+00 1.64794909e-08]     	 2.4530256296252335e-08      	 2.156580542829979e-08
 1e-08      	 	 3 	 	 [1.82167425e+01 7.19397512e+00 6.72304086e-08]     	 2.873412847280544e-07      	 2.769787401068656e-07
 1e-07      	 	 3 	 	 [1.82167425e+01 7.19397516e+00 5.31594832e-08]     	 1.4517904709995478e-06      	 1.148501214455034e-06
 1e-06      	 	 3 	 	 [1.82167393e+01 7.19397438e+00 1.23063560e-06]     	 1.6187937101639972e-05      	 1.3223100848050117e-05
 1e-05      	 	 3 	 	 [1.82166119e+01 7.19395850e+00 8.00689528e-05]     	 0.00028200995826593877      	 0.00027570851975976403
 0.0001      	 	 3 	 	 [1.82159972e+01 7.19534427e+00 5.00826228e-04]     	 0

## Finite Difference Noise

In [27]:

experiment_name = "linear_nonunique_1"
u,x,t,formula = experiment_data(n_samples=100,experiment_name=experiment_name)
dx=x[1]-x[0]
u_noise = u
ux_noise = ps.FiniteDifference(order=10,d=1, axis=0, drop_endpoints=False)._differentiate(u_noise, dx)
u_flat_noise, u_x_flat_noise = u_noise.flatten(), ux_noise.flatten()
g_noise = np.concatenate([u_flat_noise.reshape(len(u_flat_noise),1), u_x_flat_noise.reshape(len(u_flat_noise),1)], axis=1)
print(g_noise.shape)
print(f"Matrix rank = {matrix_rank(g_noise)}, svd = {svd(g_noise, compute_uv=False)}")

(10000, 2)
Matrix rank = 1, svd = [2.28823062e+04 3.02436615e-08]


In [28]:
matrix.shape

(4, 3)

In [110]:
print(matrix)

[[ 9.52816513 -0.86153446 -2.546406  ]
 [-5.74752964  2.88366887  8.52315494]
 [-7.11123871  1.40931187  4.16545171]]


In [111]:
print(matrix_noise)

[[ 9.73985977 -1.92402838 -1.97164702]
 [-5.57282292  2.91257473  9.36994399]
 [-7.04261599  1.62266081  5.10828879]]


In [112]:
noise

array([[ 0.21169464, -1.06249392,  0.57475898],
       [ 0.17470672,  0.02890585,  0.84678905],
       [ 0.06862271,  0.21334894,  0.94283708]])